# Apache Spark : Analyse des données climatiques mondiales

## Objectif :
Analyser les tendances climatiques mondiales à l'aide de Spark, y compris le nettoyage des données, l'EDA et l'extraction d'informations.

### Jeu de données / Base de Données:
**Global Surface Summary of the Day (GSOD)** provenant de NOAA.

---
##  Exploration du jeu de données


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Load CSV from HDFS") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://127.0.0.1:9866") \
    .getOrCreate()

In [ ]:
import os
import requests
import tarfile
from io import BytesIO

BASE_URL = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/archive/"

# dossier local contenant les données NOAA
OUTPUT_DIR = "/home/jovyan/work/data/noaa_gsod"
os.makedirs(OUTPUT_DIR, exist_ok=True)

for year in range(1929, 2026):
    print(f"📁 Téléchargement de l'archive {year}.tar.gz")

    archive_url = f"{BASE_URL}{year}.tar.gz"
    response = requests.get(archive_url)

    if response.status_code != 200:
        print(f"⚠️ Archive {year} introuvable, passage à l'année suivante.")
        continue

    # Création du dossier de l'année
    year_folder = os.path.join(OUTPUT_DIR, str(year))
    os.makedirs(year_folder, exist_ok=True)

    # Extraction du tar.gz en mémoire
    tar_bytes = BytesIO(response.content)

    try:
        with tarfile.open(fileobj=tar_bytes, mode="r:gz") as tar:
            tar.extractall(path=year_folder)
    except Exception as e:
        print(f"❌ Erreur lors de l'extraction de {year}.tar.gz : {e}")
        continue
 

In [7]:
df = spark.read.csv("data/noaa_gsod/*.csv", header=True, inferSchema=True)

df.show(5)
df.printSchema()

Py4JJavaError: An error occurred while calling o68.csv.
: java.net.ConnectException: Call From 1b6621aa19ea/172.23.0.7 to localhost:9866 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:828)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1616)
	at org.ap

**Questions :**
1. Combien d'enregistrements contient le jeu de données ?
2. Quels sont les noms et types de colonnes ?
3. Combien d'années de données au total ?
4. Combien de stations météorologiques uniques sont présentes ?

---
## Nettoyage des données


**Questions :**
1. Quelles colonnes ont le plus de valeurs manquantes ?
2. Après nettoyage, combien d'enregistrements restent ?

---
##  Transformation des données


In [ ]:
# Transformer le jeu de données
# Ajouter de nouvelles colonnes pour l'année, le mois, le jour
df_transformed = df_cleaned.withColumn('year', F.year(F.col('date')))
# Classifier les jours chauds
df_transformed = df_transformed.withColumn('is_hot', F.col('temperature') > 30)

**Questions :**
1. Quels sont les 5 premiers enregistrements après le parsing de la colonne `date` ?
2. Combien de jours ont été classés comme 'chauds' ?
3. Creez un graphe qui permet de visualiser le nombres d'occurences de jours 'chauds' a travers les années 

---
## Analyse exploratoire des données (EDA)


In [ ]:
# Calculer les moyennes annuelles et autres statistiques
# avg_temp_by_year = df_transformed.groupBy('year').avg('temperature')
avg_temp_by_year.show()

**Questions :**
1. Quelle est la température moyenne mondiale pour 2020 ?
2. Quelles sont les 5 stations ayant enregistré les températures moyennes les plus élevées, et où sont-elles situées ?
3. Comment les précipitations mondiales ont-elles évolué au cours des 50 dernières années, à l’échelle mondiale, puis spécifiquement en France ?

---
## Requêtes avancées


In [ ]:
# Enregistrer comme vue SQL temporaire et exécuter des requêtes
# df_transformed.createOrReplaceTempView('climate')
# spark.sql('SELECT ...')

**Questions :**
1. Quelle a été l'année la plus froide enregistrée, et quelle était la température moyenne ?
2. Quelle station a contribué avec le plus grand nombre d'enregistrements ?
3. Fournissez la requête SQL utilisée pour trouver les réponses ci-dessus.

---
## Visualisation


In [ ]:
# Exporter les données vers Pandas et créer des visualisations
# import matplotlib.pyplot as plt
# df_pandas = avg_temp_by_year.toPandas()
# plt.plot(df_pandas['year'], df_pandas['avg_temperature'])

**Questions :**
1. Créez un graphe pour visualiser les températures moyennes mondiales.
2. Créez un graphe pour visualiser les précipitations mondiales.
3. Quelles analyses supplémentaires recommanderiez-vous en fonction des visualisations, et pourriez-vous isoler les effets du réchauffement climatique ?
4. En prenant l’ensemble du jeu de données comme référence, calculez l'augmentation de la température en pourcentage par rapport à la moyenne mondiale. Ensuite, créez un graphique montrant pour chaque pays le pourcentage d'augmentation par rapport à la moyenne mondialenée